In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
import ast
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
from keras import Model
from keras import optimizers
from keras.legacy import interfaces
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D, BatchNormalization
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.optimizers import Adam

from keras.optimizers import SGD
from keras import callbacks
from data import *
from keras_helper import *

from keras.applications import InceptionV3
from keras.applications.inception_v3 import preprocess_input

Using TensorFlow backend.


In [2]:
TOT = 49673580
EPOCHS = 50
size = 75
batchsize = 640
lw = 2
channel = 1
STEPS = TOT / EPOCHS / batchsize / 3
NCATS = 340

In [3]:
# def preprocess_input(x):
#     x /= 127.5
#     x -= 1.
#     return x

In [4]:
K.clear_session()
# base_model = MobileNet(input_shape=(size, size, 3), alpha=1., weights='imagenet', include_top=False)
# x = base_model.output
# # x = GlobalAveragePooling2D()(x)
# x = Flatten()(x)
# x = Dropout(0.5)(x)

# x = Dense(512)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Dropout(0.5)(x)

# x = Dense(64)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Dropout(0.5)(x)

# predictions = Dense(NCATS, activation='softmax')(x)

# model = Model(inputs=base_model.input, outputs=predictions)

model = InceptionV3(input_shape=(size, size, 1), weights=None, classes=NCATS)

model.compile(optimizer=Adam(lr=0.002), loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 75, 75, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 37, 37, 32)   288         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 37, 37, 32)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 37, 37, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [5]:
valid_df = pd.read_csv('../input/valid.csv')
x_valid = df_to_image_array_xd(valid_df, size, lw=lw, preprocess_input=preprocess_input, channel=channel)
y_valid = keras.utils.to_categorical(valid_df.word, num_classes=NCATS)

In [6]:
print(x_valid.shape, y_valid.shape)
print('Validation array memory {:.2f} GB'.format(x_valid.nbytes / 1024.**3 ))

(34000, 75, 75, 1) (34000, 340)
Validation array memory 0.71 GB


In [7]:
train_datagen = image_generator_xd(size=size, batchsize=batchsize, lw=lw, 
                                   preprocess_input=preprocess_input,
                                  channel=channel)

In [8]:
model_prefix = 'inceptionv3_raw_head'
fold = 9
callbks = [
    ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5,
                      min_delta=0.005, mode='max', cooldown=3, verbose=1),
    callbacks.ModelCheckpoint("./models/{}{}.model".format(model_prefix, size),
                                monitor='val_categorical_accuracy', 
                                mode = 'max', save_best_only=True, verbose=1),
    callbacks.TensorBoard(log_dir='./log/{}{}'.format(model_prefix, size)),
]

In [9]:
hists = []
hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbks
)

Epoch 1/50
518/517 [==============================] - 171s 330ms/step - loss: 3.6799 - categorical_crossentropy: 3.6799 - categorical_accuracy: 0.2168 - top_3_accuracy: 0.3767 - val_loss: 4.0579 - val_categorical_crossentropy: 4.0579 - val_categorical_accuracy: 0.2261 - val_top_3_accuracy: 0.4190

Epoch 00001: val_categorical_accuracy improved from -inf to 0.22615, saving model to ./models/inceptionv3_raw_head75.model
Epoch 2/50
518/517 [==============================] - 156s 302ms/step - loss: 2.4380 - categorical_crossentropy: 2.4380 - categorical_accuracy: 0.4256 - top_3_accuracy: 0.6424 - val_loss: 9.6372 - val_categorical_crossentropy: 9.6372 - val_categorical_accuracy: 0.0348 - val_top_3_accuracy: 0.1256

Epoch 00002: val_categorical_accuracy did not improve from 0.22615
Epoch 3/50
518/517 [==============================] - 158s 305ms/step - loss: 2.0419 - categorical_crossentropy: 2.0419 - categorical_accuracy: 0.5095 - top_3_accuracy: 0.7222 - val_loss: 2.2756 - val_categorical

Epoch 22/50
518/517 [==============================] - 156s 301ms/step - loss: 1.2895 - categorical_crossentropy: 1.2895 - categorical_accuracy: 0.6808 - top_3_accuracy: 0.8520 - val_loss: 1.3260 - val_categorical_crossentropy: 1.3260 - val_categorical_accuracy: 0.6686 - val_top_3_accuracy: 0.8440

Epoch 00022: val_categorical_accuracy did not improve from 0.68503
Epoch 23/50
518/517 [==============================] - 155s 300ms/step - loss: 1.2986 - categorical_crossentropy: 1.2986 - categorical_accuracy: 0.6787 - top_3_accuracy: 0.8504 - val_loss: 1.2835 - val_categorical_crossentropy: 1.2835 - val_categorical_accuracy: 0.6780 - val_top_3_accuracy: 0.8481

Epoch 00023: val_categorical_accuracy did not improve from 0.68503
Epoch 24/50
518/517 [==============================] - 154s 297ms/step - loss: 1.3095 - categorical_crossentropy: 1.3095 - categorical_accuracy: 0.6752 - top_3_accuracy: 0.8488 - val_loss: 1.3253 - val_categorical_crossentropy: 1.3253 - val_categorical_accuracy: 0.6

Epoch 43/50
518/517 [==============================] - 153s 294ms/step - loss: 1.0851 - categorical_crossentropy: 1.0851 - categorical_accuracy: 0.7291 - top_3_accuracy: 0.8830 - val_loss: 1.0769 - val_categorical_crossentropy: 1.0769 - val_categorical_accuracy: 0.7326 - val_top_3_accuracy: 0.8819

Epoch 00043: val_categorical_accuracy did not improve from 0.73274
Epoch 44/50
518/517 [==============================] - 153s 295ms/step - loss: 1.0900 - categorical_crossentropy: 1.0900 - categorical_accuracy: 0.7276 - top_3_accuracy: 0.8818 - val_loss: 1.0821 - val_categorical_crossentropy: 1.0821 - val_categorical_accuracy: 0.7296 - val_top_3_accuracy: 0.8801

Epoch 00044: val_categorical_accuracy did not improve from 0.73274
Epoch 45/50
518/517 [==============================] - 153s 295ms/step - loss: 1.0726 - categorical_crossentropy: 1.0726 - categorical_accuracy: 0.7319 - top_3_accuracy: 0.8845 - val_loss: 1.2518 - val_categorical_crossentropy: 1.2518 - val_categorical_accuracy: 0.6

In [10]:
model.load_weights('./models/{}{}.model'.format(model_prefix, size))

In [11]:
test = pd.read_csv('../input/test_simplified.csv')
test.head()
x_test = df_to_image_array_xd(test, size, lw=lw, 
                              preprocess_input=preprocess_input,
                             channel=channel)
print(test.shape, x_test.shape)
print('Test array memory {:.2f} GB'.format(x_test.nbytes / 1024.**3 ))

(112199, 3) (112199, 75, 75, 1)
Test array memory 2.35 GB


In [12]:
test_predictions = model.predict(x_test, batch_size=128, verbose=1)

top3 = preds2catids(test_predictions)
top3.head()
top3.shape

112199/112199 [==============================] - 25s 221us/step


(112199, 3)

In [13]:
np_classes = np.load('../input/classes.npy')
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(np_classes)}
top3cats = top3.replace(id2cat)
top3cats.head()
top3cats.shape

(112199, 3)

In [14]:
test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
submission = test[['key_id', 'word']]
submission.head()

,key_id,word
0,9000003627287624,radio stereo alarm_clock
1,9000010688666847,hockey_puck steak pool
2,9000023642890129,The_Great_Wall_of_China castle fence
3,9000038588854897,mountain tent The_Eiffel_Tower
4,9000052667981386,fireplace campfire crown


In [15]:
import kaggle_util
kaggle_util.save_result(submission, 
                        '../result/{}{}.csv'.format(model_prefix, size), 
                        'quickdraw-doodle-recognition', 
                        send=True, index=False)

save result
upload result
cmd: kaggle competitions submit -c quickdraw-doodle-recognition -f ../result/inceptionv3_raw_head75.csv.7z -m "submit"
